  ##                      emotion classification using linear models

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cd drive/My Drive/Colab Notebooks/vg/NLP/

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/vg/NLP/'
/content/drive/My Drive/Colab Notebooks/vg/NLP


In [9]:
!pwd

/content/drive/My Drive/Colab Notebooks/vg/NLP


In [10]:
import pandas as pd
import numpy as np
 
import nltk
from nltk.corpus import stopwords

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from gensim.models import FastText


from contractions import contraction_map
import string,re

from nltk.stem import WordNetLemmatizer


In [11]:
from keras.layers import Dense, Embedding, Bidirectional, SpatialDropout1D,Input, TimeDistributed,Flatten, LSTM
from keras.models import Sequential,save_model,Model
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf

### Data cleaning

In [12]:
data = pd.read_csv('datasets/crowdflower-sentiment-analysis-in-text/text_emotion.csv')

In [13]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [14]:
data.sentiment.value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [15]:
df_temp = data.copy()

In [16]:
df_temp.dtypes

tweet_id      int64
sentiment    object
author       object
content      object
dtype: object

In [17]:
# Dropping rows with other emotion labels
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'boredom'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'enthusiasm'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'empty'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'fun'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'relief'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'surprise'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'love'].index)
df_temp = df_temp.drop(df_temp[df_temp.sentiment == 'hate'].index)

In [18]:
df_temp.sentiment.value_counts()

neutral      8638
worry        8459
happiness    5209
sadness      5165
anger         110
Name: sentiment, dtype: int64

In [19]:
df_temp

,tweet_id,sentiment,author,content
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
...,...,...,...,...
39992,1753918881,neutral,_Alectrona_,@jasimmo Ooo showing of your French skills!! l...
39993,1753918892,neutral,bushidosan,"@sendsome2me haha, yeah. Twitter has many uses..."
39994,1753918900,happiness,courtside101,Succesfully following Tayla!!
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor


In [20]:
df_temp = df_temp.drop('author',axis=1)
df_temp

,tweet_id,sentiment,content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
...,...,...,...
39992,1753918881,neutral,@jasimmo Ooo showing of your French skills!! l...
39993,1753918892,neutral,"@sendsome2me haha, yeah. Twitter has many uses..."
39994,1753918900,happiness,Succesfully following Tayla!!
39995,1753918954,neutral,@JohnLloydTaylor


In [21]:
df_temp.reset_index(drop=True,inplace=True)

In [22]:
df_temp

,tweet_id,sentiment,content
0,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
1,1956967696,sadness,Funeral ceremony...gloomy friday...
2,1956968416,neutral,@dannycastillo We want to trade with someone w...
3,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...
4,1956968487,sadness,"I should be sleep, but im not! thinking about ..."
...,...,...,...
27576,1753918881,neutral,@jasimmo Ooo showing of your French skills!! l...
27577,1753918892,neutral,"@sendsome2me haha, yeah. Twitter has many uses..."
27578,1753918900,happiness,Succesfully following Tayla!!
27579,1753918954,neutral,@JohnLloydTaylor


In [23]:
df_temp.shape

(27581, 3)

In [24]:
df_temp.drop('tweet_id',axis=1,inplace=True)

In [25]:
df_temp.columns = ['sentiment','text']

In [26]:
df_temp.sentiment.value_counts()

neutral      8638
worry        8459
happiness    5209
sadness      5165
anger         110
Name: sentiment, dtype: int64

In [27]:
class_count_0 , class_count_1, class_count_2, class_count_3, class_count_4 = df_temp.sentiment.value_counts()

In [28]:
class_count_anger_updated = df_temp[df_temp['sentiment'] == 'anger'].sample(class_count_3,replace=True)

In [29]:
class_count_anger_updated.reset_index(drop=True,inplace=True)

In [30]:
class_count_neutral = df_temp[df_temp['sentiment'] == 'neutral']
class_count_worry = df_temp[df_temp['sentiment'] == 'worry']
class_count_happiness = df_temp[df_temp['sentiment'] == 'happiness']
class_count_sadness= df_temp[df_temp['sentiment'] == 'sadness']

In [31]:
df = pd.concat([class_count_neutral,class_count_worry,class_count_happiness,class_count_sadness,class_count_anger_updated],axis=0) 

In [32]:
df.sentiment.value_counts()

neutral      8638
worry        8459
happiness    5209
sadness      5165
anger        5165
Name: sentiment, dtype: int64

In [33]:
df

,sentiment,text
2,neutral,@dannycastillo We want to trade with someone w...
8,neutral,cant fall asleep
17,neutral,No Topic Maps talks at the Balisage Markup Con...
25,neutral,@cynthia_123 i cant sleep
26,neutral,I missed the bl***y bus!!!!!!!!
...,...,...
5160,anger,@brainofdane DUDE. You're a hax0r!!!1! You s...
5161,anger,Confuzzled
5162,anger,"Wow, I'm really bored... and pissed I left my ..."
5163,anger,The &quot;Catch Me If You Can&quot; DVD that I...


In [34]:
df.reset_index(drop=True,inplace=True)

In [35]:
df

,sentiment,text
0,neutral,@dannycastillo We want to trade with someone w...
1,neutral,cant fall asleep
2,neutral,No Topic Maps talks at the Balisage Markup Con...
3,neutral,@cynthia_123 i cant sleep
4,neutral,I missed the bl***y bus!!!!!!!!
...,...,...
32631,anger,@brainofdane DUDE. You're a hax0r!!!1! You s...
32632,anger,Confuzzled
32633,anger,"Wow, I'm really bored... and pissed I left my ..."
32634,anger,The &quot;Catch Me If You Can&quot; DVD that I...


### Data pre-processing

##### Stopword Exclusion

In [36]:
#Using Sentiment lexicons to be excluded from stopwords

In [37]:
df_pos = pd.read_csv('./lexicons/positive.csv')
df_neg = pd.read_csv('./lexicons/negative.csv')

### Helper functions 

In [38]:
def expand_text(text):
    text = text.lower()
    text = text.replace("`","'")
    
    #Expand Contractions
    contraction_dict = contraction_map
    contraction_keys = list(contraction_dict.keys())
    
    for word in text.split():
        if word in contraction_keys:
            text = text.replace(word, contraction_dict[word])
        else:
            continue
    
    return text    

In [39]:
def clean_text(text):
    text = text.translate(string.punctuation)
    text = text.lower().split()
    
    df_pos_words = list(df_pos.words)
    df_neg_words = list(df_neg.words)
    
    positive = []
    for i in range(0,len(df_pos_words)):
        positive.append(df_pos_words[i].lower().replace(" ",""))
        
    negative = []
    for i in range(0,len(df_neg_words)):
        negative.append(df_neg_words[i].lower().replace(" ",""))
        
    pos_set = set(positive)
    neg_set = set(negative)
    
    keywords = set(["above","and","below","not"])
    
    keywords.update(pos_set)
    keywords.update(neg_set)
    
    stopwords_set = set(stopwords.words('english'))
    stops = stopwords_set - keywords
    
    
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    text = re.sub(r"[^A-Za-z0-9^,!./\'+-=]"," ",text)
    text = re.sub(r"what's","what is",text)
    text = re.sub(r"\'s"," ",text)
    text = re.sub(r"\'ve"," have ",text)
    text = re.sub(r"n't"," not ",text)
    text = re.sub(r"i'm"," i am ",text)
    text = re.sub(r"\'re"," are ",text)
    text = re.sub(r"\'d", " would ",text)
    text = re.sub(r"\'ll", " will ",text)
    text = re.sub(r","," ",text)
    text = re.sub(r"\."," ",text)
    text = re.sub(r"!"," ! ",text)
    text = re.sub(r"\/"," ",text)
    text = re.sub(r"\^"," ^ ",text)
    text = re.sub(r"\+"," + ",text)
    text = re.sub(r"\-"," - ",text)
    text = re.sub(r"\="," = ",text)
    text = re.sub(r"'"," ",text)
    text = re.sub(r"(\d+)(k)",r"\g<1>000",text)
    text = re.sub(r":", " : ",text)
    text = re.sub(r" e g "," eg ",text)
    text = re.sub(r"b g "," bg ",text)
    text = re.sub(r" u s "," american ",text)
    text = re.sub(r"\0s","0",text)
    text = re.sub(r"e - mail","email",text)
    text = re.sub(r"\s{2,}"," ",text)
    
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    text = " ".join(lemmatized_words)
    
    return text

In [40]:
df

,sentiment,text
0,neutral,@dannycastillo We want to trade with someone w...
1,neutral,cant fall asleep
2,neutral,No Topic Maps talks at the Balisage Markup Con...
3,neutral,@cynthia_123 i cant sleep
4,neutral,I missed the bl***y bus!!!!!!!!
...,...,...
32631,anger,@brainofdane DUDE. You're a hax0r!!!1! You s...
32632,anger,Confuzzled
32633,anger,"Wow, I'm really bored... and pissed I left my ..."
32634,anger,The &quot;Catch Me If You Can&quot; DVD that I...


In [41]:
df['text'] = df['text'].progress_apply(lambda x : expand_text(x))

In [42]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [43]:
df['text'] = df['text'].progress_apply(lambda x: clean_text(x))

In [44]:
df

,sentiment,text
0,neutral,dannycastillo want trade someone houston ticke...
1,neutral,cant fall asleep
2,neutral,topic map talk balisage markup conference 2009...
3,neutral,cynthia 123 cant sleep
4,neutral,missed bl y bus ! ! ! ! ! ! ! !
...,...,...
32631,anger,brainofdane dude hax0r ! ! ! 1 ! put final cut...
32632,anger,confuzzled
32633,anger,wow really bored and pissed left library card ...
32634,anger,quot;catch can quot; dvd rented blockbuster co...


In [45]:
df = df.sample(frac=1)

In [46]:
df.reset_index(drop=True,inplace=True)

### create Tokenizer

In [47]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
vocab_size = len(tokenizer.word_index)+1
print(vocab_size)

34512


In [48]:
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences,maxlen=50)

## Use word embeddings

In [49]:
embeddings_index = {}
f = open('./embeddings/wiki-news-300d-1M-subword.vec',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype="float32")
    embeddings_index[word] = coefs
f.close()

## Create a weight matrix 

In [50]:
embedding_matrix = np.zeros((vocab_size,300))
for word,index in tokenizer.word_index.items():
    if index > vocab_size -1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 

## Model building 

In [51]:
max_len = 50

In [58]:
es = EarlyStopping(monitor='val_loss',verbose=1,patience=10)
mc = ModelCheckpoint('./saved_models/best_model.h5',save_best_only=True,verbose=1)

In [54]:
df['sentiment'].value_counts()

neutral      8638
worry        8459
happiness    5209
sadness      5165
anger        5165
Name: sentiment, dtype: int64

Test layers

In [81]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
model_lstm.add(LSTM(128,dropout=0.5))
model_lstm.add(Dense(100,activation='relu'))
model_lstm.add(Dense(100,activation='relu'))
model_lstm.add(Dense(5,activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [82]:
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 50, 300)           10353600  
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_11 (Dense)             (None, 100)               12900     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 505       
Total params: 10,596,753
Trainable params: 243,153
Non-trainable params: 10,353,600
_________________________________________________________________


In [83]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = encoder.fit_transform(df['sentiment'])
labels = to_categorical(labels)

In [84]:
model_lstm.fit(data,labels,validation_split=0.1,epochs=10,callbacks=[es,mc])

Train on 29372 samples, validate on 3264 samples
Epoch 1/10
29372/29372 [==============================] - 92s 3ms/step - loss: 1.4563 - accuracy: 0.3678 - val_loss: 1.2617 - val_accuracy: 0.4660

Epoch 00001: val_loss did not improve from 0.95392
Epoch 2/10
29372/29372 [==============================] - 92s 3ms/step - loss: 1.2714 - accuracy: 0.4553 - val_loss: 1.1163 - val_accuracy: 0.5156

Epoch 00002: val_loss did not improve from 0.95392
Epoch 3/10
29372/29372 [==============================] - 92s 3ms/step - loss: 1.1702 - accuracy: 0.4908 - val_loss: 1.0480 - val_accuracy: 0.5365

Epoch 00003: val_loss did not improve from 0.95392
Epoch 4/10
29372/29372 [==============================] - 91s 3ms/step - loss: 1.1031 - accuracy: 0.5145 - val_loss: 1.0152 - val_accuracy: 0.5463

Epoch 00004: val_loss did not improve from 0.95392
Epoch 5/10
29372/29372 [==============================] - 92s 3ms/step - loss: 1.0719 - accuracy: 0.5231 - val_loss: 0.9963 - val_accuracy: 0.5515

Epoch 0

In [92]:
txt = ["India Overtakes Russia As Third Worst-Hit Nation In COVID-19 Tally "]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_len)
pred = model_lstm.predict(padded)
labels = ['anger',' happiness','neutral',' sadness','worry']
print(pred, labels[np.argmax(pred)])

[[1.3059891e-05 4.3242693e-02 2.5437132e-01 2.3090135e-01 4.7147152e-01]] worry


In [93]:
fp = './saved_models/best_model.h5'
save_model(model_lstm,fp,include_optimizer=True)